In [1]:
#### now with propagation
""" this scripts run the analysis for comparing QTL studies, tissues together with therapy areas matched"""

from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)

spark = SparkSession.builder.getOrCreate()

platform_v = "24.06"

target_path = (
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/targets/"
)
target = spark.read.parquet(target_path)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/01 10:53:44 INFO SparkEnv: Registering MapOutputTracker
24/08/01 10:53:44 INFO SparkEnv: Registering BlockManagerMaster
24/08/01 10:53:44 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/08/01 10:53:44 INFO SparkEnv: Registering OutputCommitCoordinator


spark session created at 2024-08-01 10:53:51.493456
Analysis started on 2024-08-01 at  2024-08-01 10:53:51.493456


24/08/01 10:54:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
target.filter(F.col("tep").isNotNull()).select("id", "tep").show(truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id             |tep                                                                                                                                                             |
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ENSG00000171303|{TASK1, Human TWIK-Related Acid-Sensitive K+ Channel 1 (TASK1), Neurological Disorders, https://www.thesgc.org/tep/task1}                                       |
|ENSG00000140876|{NUDT7, Human Peroxisomal Coenzyme A Diphosphatase NUDT7 (NUDT7), Metabolic diseases, https://www.thesgc.org/tep/nudt7}                                         |
|ENSG00000143379|{SETDB1, Human SET domain bifurcated 1 (SETDB1), Tudor domain, Oncology, https://www.the

In [ ]:
target.show

In [28]:
#### now with propagation
""" this scripts run the analysis for comparing QTL studies, tissues together with therapy areas matched"""

from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)

spark = SparkSession.builder.getOrCreate()

platform_v = "24.06"

target_path = (
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/targets/"
)
target = spark.read.parquet(target_path)

disease_path = (
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/diseases/"
)
diseases = spark.read.parquet(disease_path)

evidences = spark.read.parquet(
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/evidence"
)

coloc = spark.read.parquet(
    "gs://genetics-portal-dev-data/22.09.1/outputs/v2d_coloc"
).filter(F.col("right_type") != "gwas")


### laod sample sizes - look at the document
sampleSize = spark.read.csv("gs://ot-team/jroldan/colocSampleSize.csv", header=True)

terminated = terminated_td(
    spark, "gs://ot-team/jroldan/analysis/targetDiseaseStoppedNegative.csv"
)
### load QTL tissues mapped to therapy areas
onto_samples = spark.read.csv(
    "gs://ot-team/jroldan/20240112_mappedSQLtissuesGTP3_5.csv", header=True
)

#### GSEA annotation for hallmark inflamation targets
immflam_annot = (
    spark.read.json(
        "gs://ot-team/jroldan/analysis/HALLMARK_INFLAMMATORY_RESPONSE.v2023.2.Hs_edited.json"
    )
    .select(F.explode_outer("geneSymbols").alias("approvedSymbol"))
    .withColumn("isInflam", F.lit("yes"))
)

#### Build the uniqueBetas_analyse
targetType = (
    target.select("id", "approvedSymbol", F.explode_outer("targetClass"))
    .select("id", "approvedSymbol", "col.label")
    .groupBy("id", "approvedSymbol")
    .agg(F.collect_set("label").alias("label"))
    .filter(F.col("label").isNotNull())
    .selectExpr(
        "id as targetIdtargetType",
        "approvedSymbol as approvedSymbol",
        "label as targetType",
    )
    .join(immflam_annot, on="approvedSymbol", how="left")
)

### take ontology of samples
samplesOnto = (
    onto_samples.withColumn(
        "right_bio_feature", F.split(F.col("original"), " - ").getItem(0)
    )
    .withColumn(
        "therapyArea", F.split(F.col("20231207_curated_simplified"), " - ").getItem(2)
    )
    .withColumn("EFO", F.split(F.col("20231207_curated_simplified"), " - ").getItem(1))
    .withColumn(
        "right_bio_feature2",
        F.split(F.col("20231207_curated_simplified"), " - ").getItem(0),
    )
    .drop(
        "curated_simplified",
        "20231207_curated_simplified",
        "original",
        "curated",
        "_c3",
        "_c4",
    )
)

coloc2 = (
    coloc.select(
        F.concat_ws("_", "left_chrom", "left_pos", "left_ref", "left_alt").alias(
            "left_locus_id"
        ),
        F.concat_ws("_", "right_chrom", "right_pos", "right_ref", "right_alt").alias(
            "right_locus_id"
        ),
        F.col("left_study").alias("left_study_id"),
        F.col("right_study").alias("right_study_id"),
        "right_gene_id",
        "coloc_h4",
        "left_var_right_study_beta",
        "right_phenotype",
        F.col("left_type"),
        F.col("right_type"),
        F.col("right_bio_feature"),
        F.col("is_flipped"),
        "left_var_right_study_pval",
    )
    .withColumn(
        "beta_assessed",  ### diferent from sQTL and oQTL
        F.when(
            (F.col("left_var_right_study_beta") > 0)
            & (F.col("right_study_id") != "GTEx-sQTL"),
            F.lit("gof"),
        ).when(
            (F.col("left_var_right_study_beta") < 0)
            & (F.col("right_study_id") != "GTEx-sQTL"),
            F.lit("lof"),
        )
        #### for sQTL is the opposite
        .when(
            (F.col("left_var_right_study_beta") > 0)
            & (F.col("right_study_id") == "GTEx-sQTL"),
            F.lit("lof"),
        )
        .when(
            (F.col("left_var_right_study_beta") < 0)
            & (F.col("right_study_id") == "GTEx-sQTL"),
            F.lit("gof"),
        )
        .otherwise(F.lit("neutral")),
    )
    .join(samplesOnto, on=["right_bio_feature"], how="left")
    .drop("right_bio_feature")
)
### check for disparities using count of different assessment for beta for target

disparities = coloc2.groupBy("left_locus_id", "left_study_id", "right_gene_id").agg(
    F.size(F.collect_set("beta_assessed")).alias("count"),
)
### add the label of which left_locus_id,left_study_id and right_gene_id are having contradictions

coloc3 = (
    coloc2.withColumnRenamed("right_bio_feature2", "right_bio_feature")
    .join(
        disparities, on=["left_locus_id", "left_study_id", "right_gene_id"], how="left"
    )
    .persist()
)
#### Run directionOfEffect
prueba_assessment = directionOfEffect(evidences, platform_v)

## add therapeuticArea name to  diseases
taDf = spark.createDataFrame(
    data=[
        ("MONDO_0045024", "cell proliferation disorder", "Oncology"),
        ("EFO_0005741", "infectious disease", "Other"),
        ("OTAR_0000014", "pregnancy or perinatal disease", "Other"),
        ("EFO_0005932", "animal disease", "Other"),
        ("MONDO_0024458", "disease of visual system", "Other"),
        ("EFO_0000319", "cardiovascular disease", "Other"),
        ("EFO_0009605", "pancreas disease", "Other"),
        ("EFO_0010282", "gastrointestinal disease", "Other"),
        ("OTAR_0000017", "reproductive system or breast disease", "Other"),
        ("EFO_0010285", "integumentary system disease", "Other"),
        ("EFO_0001379", "endocrine system disease", "Other"),
        ("OTAR_0000010", "respiratory or thoracic disease", "Other"),
        ("EFO_0009690", "urinary system disease", "Other"),
        ("OTAR_0000006", "musculoskeletal or connective tissue disease", "Other"),
        ("MONDO_0021205", "disease of ear", "Other"),
        ("EFO_0000540", "immune system disease", "Other"),
        ("EFO_0005803", "hematologic disease", "Other"),
        ("EFO_0000618", "nervous system disease", "Other"),
        ("MONDO_0002025", "psychiatric disorder", "Other"),
        ("OTAR_0000020", "nutritional or metabolic disease", "Other"),
        ("OTAR_0000018", "genetic, familial or congenital disease", "Other"),
        ("OTAR_0000009", "injury, poisoning or other complication", "Other"),
        ("EFO_0000651", "phenotype", "Other"),
        ("EFO_0001444", "measurement", "Other"),
        ("GO_0008150", "biological process", "Other"),
    ],
    schema=StructType(
        [
            StructField("taId", StringType(), True),
            StructField("taLabel", StringType(), True),
            StructField("taLabelSimple", StringType(), True),
        ]
    ),
)
#### prepare for disease propagation, but explode at the end
diseases2 = (
    diseases.withColumn("taId", F.explode_outer("therapeuticAreas"))
    .join(taDf.drop("taLabelSimple"), on="taId", how="left")
    .groupBy("id", "parents", "name")
    .agg(
        F.collect_set("taId").alias("taId"),
        F.collect_set("taLabel").alias("diseaseTherapyAreas"),
    )
)
#### load data from ot genetics with the assessments
delimiter = ","
evidence_ot_genetics = (
    (
        prueba_assessment.filter((F.col("datasourceId").isin(["ot_genetics_portal"])))
        .groupBy("targetId", "diseaseId", "variantId", "studyId")
        .pivot("directionOnTrait")
        .count()
        .drop("noEvaluable", "conflict/noEvaluable")
        .persist()
    )
    .join(diseases2, F.col("diseaseId") == diseases2.id, "left")
    .withColumnRenamed("diseaseId", "oldDiseaseId")
    .withColumn(
        "diseaseId",
        F.explode_outer(F.concat(F.array(F.col("id")), F.col("parents"))),
    )
    .drop("id")
)

coloc_otgene = (
    coloc3.withColumnRenamed("left_study_id", "studyId")
    .withColumnRenamed("left_locus_id", "locusId")
    .withColumnRenamed("right_gene_id", "targetId")
    .join(
        evidence_ot_genetics.withColumnRenamed("variantId", "locusId"),
        on=["studyId", "locusId", "targetId"],
        how="left",
    )
    .withColumnRenamed("GoF", "GoF_OT")  # remove
    .withColumnRenamed("LoF", "LoF_OT")  # remove
).persist()

chembl_trials = (
    prueba_assessment.filter((F.col("datasourceId").isin(["chembl"])))
    .groupBy("targetId", "diseaseId")
    .agg(F.max(F.col("clinicalPhase")).alias("maxClinPhase"))
)

chembl = (
    (
        prueba_assessment.filter(
            (F.col("datasourceId").isin(["chembl"]))
            & (F.col("homogenized").isin(["noEvaluable", "dispar"]) == False)
        )
    )
    .groupBy("targetId", "diseaseId")
    .pivot("variantEffect")
    .count()
    .withColumnRenamed("LoF", "LoF_Ch")
    .withColumnRenamed("GoF", "GoF_Ch")
    .join(chembl_trials, on=["targetId", "diseaseId"], how="left")
    .persist()
)

coloc_bnch = coloc_otgene.join(chembl, on=["targetId", "diseaseId"], how="inner")
coloc_bnch2 = coloc_bnch.join(
    sampleSize.select("right_study_id", "sampleSize"), on="right_study_id", how="left"
).persist()

withEvidence = (
    coloc_bnch2.withColumn(
        "ChEMBL",
        F.when(
            (F.col("GoF_Ch").isNotNull()) & (F.col("LoF_Ch").isNotNull()),
            F.lit("gof&lof"),
        )
        .when(
            (F.col("LoF_Ch").isNotNull()) & (F.col("GoF_Ch").isNull()),
            F.lit(F.lit("lof")),
        )
        .when(
            (F.col("GoF_Ch").isNotNull()) & (F.col("LoF_Ch").isNull()),
            F.lit(F.lit("gof")),
        ),
    )
    .withColumn(
        "Coherency_chembl",
        F.when(  ### there are cases of drug with gof&lof
            (F.col("protect").isNotNull()),
            F.when(
                (F.col("beta_assessed") == "gof"),
                F.when(
                    (F.col("GoF_Ch").isNotNull()) & (F.col("LoF_Ch").isNull()),
                    F.lit("coherent"),
                ).when(
                    (F.col("LoF_Ch").isNotNull()) & (F.col("GoF_Ch").isNull()),
                    F.lit("dispar"),
                ),
            ).when(
                (F.col("beta_assessed") == "lof"),
                F.when(
                    (F.col("GoF_Ch").isNotNull()) & (F.col("LoF_Ch").isNull()),
                    F.lit("dispar"),
                ).when(
                    (F.col("LoF_Ch").isNotNull()) & (F.col("GoF_Ch").isNull()),
                    F.lit("coherent"),
                ),
            ),
        ),
    )
    .join(target.select("id", "approvedSymbol"), target.id == F.col("targetId"), "left")
    .drop("id")
    .persist()
)

uniqueBetas = withEvidence.persist()

custom_schema = StructType(
    [
        StructField("_c0", StringType(), True),
        StructField("_c1", StringType(), True),
        StructField("_c2", StringType(), True),
        StructField("_c3", DecimalType(38, 37), True),
        # Add more fields as needed
    ]
)

### windows function for adjusting pvalue using benjamini-hochberg correction
window = Window.partitionBy("studyId").orderBy("pVal")
window2 = Window.partitionBy("studyId")

### read the tissue enrichent's file
tissueEnrichment = (
    spark.read.csv(
        "gs://ot-team/jroldan/Tissue_enrichment_results/",
        sep="\t",
        schema=custom_schema,
    )
    .selectExpr(
        "_c0 as studyId",
        "_c1 as tissueEnriched",
        "_c3 as pVal",
        #### instead of removing, keeping the ones with pvalues=0 to not perturb the rank
    )
    .withColumn(
        "pVal",
        F.when(F.col("pVal") == 0e-37, F.lit(1e-37)).otherwise(F.col("pVal")),
        ### ranking column
    )
    .withColumn(
        "index",
        F.row_number().over(window),
        ### total number of rows per study column
    )
    .withColumn(
        "length",
        F.last("index").over(window2),
        ### BH adjusted p values calculation
    )
    .withColumn("adjPVal", (F.col("pVal") * F.col("length")) / F.col("index"))
)

### read the file with matching tissue enriched with therapy areas
tissueEnrichTherAreas = (
    spark.read.csv(
        "gs://ot-team/jroldan/analysis/20231207_gwasTissueEnrrichedToTherapyAreas.csv/",
        sep=",",
        header=True,
    )
    .withColumnRenamed("tissue", "tissueEnriched")
    .withColumn(
        "tissueEnrichedTherapyAreas",
        F.array(F.col("TherapyArea"), F.col("Alternative")),
    )
    .persist()
)

### prepare the file of tissue enriched with their therapy areas
studyEnrichTherArea = (
    tissueEnrichment.join(
        tissueEnrichTherAreas.select("tissueEnriched", "tissueEnrichedTherapyAreas"),
        on="tissueEnriched",
        how="left",
        ## filter by p value <0.05
    )
    ### filter by adjusted p values below 0.05
    .filter(F.col("adjPVal") < 0.05)
    .groupBy("studyId")
    .agg(
        F.array_except(
            F.flatten(F.collect_set("tissueEnrichedTherapyAreas")), F.array(F.lit(None))
        ).alias("studyTherapyArea"),
        F.collect_set("tissueEnriched").alias("tissuesEnriched"),
    )
)

#### mapping right_bio_feature - disease
schema_rbf = StructType(
    [
        StructField("remove", StringType(), True),
        StructField("right_bio_feature", StringType(), True),
        StructField("name", StringType(), True),
        StructField("matchRBFToDisease", StringType(), True),
        # Add more fields as needed
    ]
)

### read the tissue enrichent's file
tissueToDisease = (
    spark.read.csv(
        "gs://ot-team/jroldan/analysis/20240415_newcurationRBFToDisease.csv/",
        header=True,
    )
    .withColumn(  ### we included this because need to correct to avoid multiplications
        "matchRBFToDisease",
        F.when(
            (
                F.col("name") == "diabetes mellitus"
            )  ## there are yes and no for this combination, and is "no"
            & (F.col("right_bio_feature") == "DLPFC_NAIVE"),
            F.lit("no"),
        ).otherwise(F.col("matchRBFToDisease")),
    )
    .select("name", "right_bio_feature", "matchRBFToDisease")
    .distinct()
)


uniqueBetas_location = (
    (
        target_membrane(spark, target, uniqueBetas).drop(
            "mb",
            "counted",
            "HPA_membrane",
            "HPA_secreted",
            "uniprot_membrane",
            "uniprot_secreted",
            "loc",
            "location_id",
            # "result",
            "loc_id",
        )
        # .join(targetType, on=["targetId"], how="left")
    )
    .join(tissueToDisease, on=["name", "right_bio_feature"], how="left")
    .withColumn("qtlTherapyArea", F.split(F.col("therapyArea"), ","))
    .join(studyEnrichTherArea, on="studyId", how="left")
    .withColumn(
        "taIntersectQtlDisease",
        F.array_intersect(F.col("qtlTherapyArea"), F.col("diseaseTherapyAreas")),
    )
    .withColumn(
        "taIntersectQtlTissueEnriched",
        F.array_intersect(F.col("qtlTherapyArea"), F.col("studyTherapyArea")),
    )
    .withColumn(
        "taNIntersectQtlDisease",
        F.when(
            F.size(
                F.array_intersect(F.col("qtlTherapyArea"), F.col("diseaseTherapyAreas"))
            )
            >= 1,
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "taNIntersectQtlTissueEnriched",
        F.when(
            F.size(
                F.array_intersect(F.col("qtlTherapyArea"), F.col("studyTherapyArea"))
            )
            >= 1,
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .join(targetType, on="approvedSymbol", how="left")
)


"""single column comparisons"""
dfs = {}
claves = []
columns = []
list_columns = [
    "right_bio_feature",
    "right_type",
    "beta_assessed",
    "right_study_id",
    "therapyArea",
]

for x in list_columns:
    a = [data[0] for data in uniqueBetas_location.select(x).distinct().collect()]
    for y in a:
        claves.append(y)
        columns.append(x)
#### remove the ".0" that is problematic
claves2 = [str(s).replace(".0", "") for s in claves]

for key, value in zip(claves2, columns):
    dfs[key] = value

"""combined column comparisons"""
list1 = []
list2 = []
cols = uniqueBetas_location.drop("Coherency_geneBurden").columns

### make list per column to be combined
study_column = [
    str(row.right_study_id)
    for row in uniqueBetas_location.filter(F.col("right_study_id").isNotNull())
    .select("right_study_id")
    .distinct()
    .collect()
]

qtl_type = [
    str(row.right_type)
    for row in uniqueBetas_location.filter(F.col("right_type").isNotNull())
    .select("right_type")
    .distinct()
    .collect()  ## there are "Nulls" in this column
]

biofeature_column = [
    str(row.right_bio_feature)
    for row in uniqueBetas_location.filter(F.col("right_bio_feature").isNotNull())
    .select("right_bio_feature")
    .distinct()
    .collect()
]

### make a dictionary with column combinations study & qtl
for x in study_column:
    for y in qtl_type:
        list1.append(x)
        list2.append(y)

### make a dictionary with column combinations biofeature & qtl
for x in biofeature_column:
    for y in qtl_type:
        list1.append(x)
        list2.append(y)

combined_lists = [list(pair) for pair in zip(list1, list2)]


## collect all columns in an array
### drop Coherency_geneBurden because it contains "coherent" and "dispar" words
def dict_comb_comp(combined_lists, study_column, qtl_type):
    new2 = {}
    for x, n in combined_lists:
        if x in study_column:
            if n in qtl_type:
                new2.update(
                    {f"{x}&{n}": {"right_type": {f"{n}": {"right_study_id": f"{x}"}}}}
                )
    return new2


new2 = dict_comb_comp(combined_lists, study_column, qtl_type)

df_string = (
    uniqueBetas_location.drop("Coherency_geneBurden")
    .select([F.col(col_name).cast("string").alias(col_name) for col_name in cols])
    .withColumn("array", F.array(cols))
)

"""make the dataframe with column combinations"""
df_string2 = df_string.select(
    ["*"]
    + [
        F.when(
            (F.col(i) == b)
            & (F.col(c) == t)
            & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .otherwise(F.lit("no"))
        .alias(a)
        for a, x in new2.items()
        for i, z in x.items()
        for b, y in z.items()
        for c, t in y.items()
        # print(a, i, b, c, t)
    ]
).persist()

delimiter = ","  ### to convert string of propagated traits to array

uniqueBetas_analyse = (
    df_string2.select(
        ["*"]
        + [  #### only doe
            F.when(
                ((f"{x}") == F.col(c)) & (F.col("Coherency_chembl") == "coherent"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no"))
            .alias(f"{x}_{c}")
            for x, c in dfs.items()
        ]
    )
    .join(terminated, on=["targetId", "diseaseId"], how="left")
    .withColumn(
        "Phase4",
        F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase0",
        F.when(F.col("maxClinPhase") >= 0, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "allPhases",
        F.when(F.col("maxClinPhase").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "PhaseT",
        F.when(F.col("prediction").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<4",
        F.when(F.col("maxClinPhase") < 4, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=3",
        F.when(F.col("maxClinPhase") <= 3, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=2",
        F.when(F.col("maxClinPhase") <= 2, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=1",
        F.when(F.col("maxClinPhase") <= 1, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase0",
        F.when(F.col("maxClinPhase") >= 0, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "coloc>_60",
        F.when(
            (F.col("coloc_h4") >= 0.60) & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .when(
            (F.col("coloc_h4") >= 0.60) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4") < 0.60) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4").isNull()) | (F.col("Coherency_chembl").isNull()),
            F.lit("no"),
        )
        .otherwise(F.lit("no")),
    )
    .withColumn(
        "coloc>_80",
        F.when(
            (F.col("coloc_h4") >= 0.80) & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .when(
            (F.col("coloc_h4") >= 0.80) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4") < 0.80) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4").isNull()) | (F.col("Coherency_chembl").isNull()),
            F.lit("no"),
        )
        .otherwise(F.lit("no")),
    )
    .withColumn(
        "coloc>_85",
        F.when(
            (F.col("coloc_h4") >= 0.85) & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .when(
            (F.col("coloc_h4") >= 0.85) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4") < 0.85) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4").isNull()) | (F.col("Coherency_chembl").isNull()),
            F.lit("no"),
        )
        .otherwise(F.lit("no")),
    )
    .withColumn(
        "coloc>_90",
        F.when(
            (F.col("coloc_h4") >= 0.90) & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .when(
            (F.col("coloc_h4") >= 0.90) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4") < 0.90) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4").isNull()) | (F.col("Coherency_chembl").isNull()),
            F.lit("no"),
        )
        .otherwise(F.lit("no")),
    )
    .withColumn(
        "coloc>_95",
        F.when(
            (F.col("coloc_h4") >= 0.95) & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        )
        .when(
            (F.col("coloc_h4") >= 0.95) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4") < 0.95) & (F.col("Coherency_chembl") == "dispar"),
            F.lit("no"),
        )
        .when(
            (F.col("coloc_h4").isNull()) | (F.col("Coherency_chembl").isNull()),
            F.lit("no"),
        )
        .otherwise(F.lit("no")),
    )
    .withColumn(
        "secreted",
        F.when(F.col("Nr_secreted") == 1, F.lit("yes"))
        .when(F.col("Nr_secreted") == 0, F.lit("no"))
        .otherwise(None),
    )
    .withColumn(
        "matchQtlTeTherArea",
        F.when(
            (F.col("taNIntersectQtlTissueEnriched") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "matchQtlDiseaseTherArea",
        F.when(
            (F.col("taNIntersectQtlDisease") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "matchQtlRBFrelevantToDisease",
        F.when(
            (F.col("matchRBFToDisease") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doeCoherent",
        F.when(F.col("Coherency_chembl") == "coherent", F.lit("yes")).otherwise(
            F.lit("no")
        ),
    )
    .withColumn(
        "doeNotCoherent",
        F.when(F.col("Coherency_chembl") == "coherent", F.lit("yes")).otherwise(
            F.lit("no")
        ),
    )
    .withColumn(
        "doe&matchQtlTeTherArea",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("taNIntersectQtlTissueEnriched") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doe&matchQtlDiseaseTherArea",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("taNIntersectQtlDisease") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doe&matchQtlRBFrelevantToDisease",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("matchRBFToDisease") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doeInflam",
        F.when(
            (F.col("coherency_chembl") == "coherent") & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "isInflam",
        F.when(F.col("isInflam") == "yes", F.lit("yes")).otherwise(F.lit("no")),
    )
    ### columns combinations for tissue enrichment and therapy areas
    .withColumn(
        "doeInflamMatchQtlTeTherArea",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("taNIntersectQtlTissueEnriched") == "yes")
            & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doeInflamMatchQtlDiseaseTherArea",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("taNIntersectQtlDisease") == "yes")
            & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "doeInflamMatchQtlRBFrelevantToDisease",
        F.when(
            (F.col("Coherency_chembl") == "coherent")
            & (F.col("matchRBFToDisease") == "yes")
            & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    #### columns considering only inflam and tissue/therapy areas
    .withColumn(
        "inflamMatchQtlTeTherArea",
        F.when(
            (F.col("taNIntersectQtlTissueEnriched") == "yes")
            & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "inflamMatchQtlDiseaseTherArea",
        F.when(
            (F.col("taNIntersectQtlDisease") == "yes") & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "inflamMatchQtlRBFrelevantToDisease",
        F.when(
            (F.col("matchRBFToDisease") == "yes") & (F.col("isInflam") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .drop("targetType")  ### remove nonsense columns for this
    .filter(
        (F.col("coherency_chembl").isNotNull())
        & (F.col("name") != "COVID-19")
        # | (F.col("name").isNull())
    )
    .withColumn(
        "RightTissueRightDoE",
        F.when(
            (F.col("matchRBFToDisease") == "yes")
            & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "RightTissueDifDoE",
        F.when(
            (F.col("matchRBFToDisease") == "yes")
            & (F.col("Coherency_chembl") == "dispar"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "DifTissueDifDoE",
        F.when(
            (
                (F.col("matchRBFToDisease") == "no")
                | (F.col("matchRBFToDisease").isNull())
            )
            & (F.col("Coherency_chembl") == "dispar"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "DifTissueSameDoE",
        F.when(
            (F.col("matchRBFToDisease") == "no")
            | (F.col("matchRBFToDisease").isNull())
            & (F.col("Coherency_chembl") == "coherent"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "SameDoERightTissueDifDoEOutTissue",
        F.when(
            (F.col("RightTissueRightDoE") == "yes")
            & (F.col("DifTissueDifDoE") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "DifDoERightTissueSameDoEOutTissue",
        F.when(
            (F.col("RightTissueDifDoE") == "yes")
            & (F.col("DifTissueSameDoE") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .withColumn(
        "DifDoERightTissueDifDoEOutTissue",
        F.when(
            (F.col("RightTissueDifDoE") == "yes") & (F.col("DifTissueDifDoE") == "yes"),
            F.lit("yes"),
        ).otherwise(F.lit("no")),
    )
    .drop(
        "array",
        "remove",
        "Phase0",
        "allPhases",
        "Terminated",
        "propagatedTraits",
        "count",
        "clinicalStatus",
        "prediction",
        "allPhases",
        "propagatedTraits",
        "targetType",
    )
    .repartition(100)
    .persist()
)

24/07/30 20:58:26 WARN CacheManager: Asked to cache already cached data.
24/07/30 20:58:27 WARN CacheManager: Asked to cache already cached data.
24/07/30 20:58:27 WARN CacheManager: Asked to cache already cached data.
24/07/30 20:58:28 WARN CacheManager: Asked to cache already cached data.
24/07/30 20:58:29 WARN CacheManager: Asked to cache already cached data.
24/07/30 20:58:29 WARN CacheManager: Asked to cache already cached data.


In [4]:
#### now with propagation
""" this scripts run the analysis for comparing QTL studies, tissues together with therapy areas matched"""

from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)

spark = SparkSession.builder.getOrCreate()

platform_v = "24.06"

target_path = (
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/targets/"
)
target = spark.read.parquet(target_path)

disease_path = (
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/diseases/"
)
diseases = spark.read.parquet(disease_path)

evidences = spark.read.parquet(
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/evidence"
)
indications = spark.read.parquet(
    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/indication"
)

In [6]:
###
indications.printSchema()

root
 |-- id: string (nullable = true)
 |-- indications: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- disease: string (nullable = true)
 |    |    |-- efoName: string (nullable = true)
 |    |    |-- references: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- source: string (nullable = true)
 |    |    |    |    |-- ids: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- maxPhaseForIndication: double (nullable = true)
 |-- approvedIndications: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- indicationCount: integer (nullable = true)



In [9]:
evidences.filter(F.col("datasourceId") == "chembl").select("drugId").distinct().count()

4325

In [12]:
evidChembl = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .groupBy("drugId", "diseaseId")
    .agg(F.max(F.col("clinicalPhase")))
    .persist()
)

In [8]:
indications.select("id").distinct().count()

9683

In [14]:
evidChembl.show()

+-------------+-------------+------------------+
|       drugId|    diseaseId|max(clinicalPhase)|
+-------------+-------------+------------------+
|CHEMBL1201293|  EFO_0003758|               1.0|
| CHEMBL445813|  EFO_0000616|               1.0|
| CHEMBL574737|  EFO_0000702|               2.0|
|   CHEMBL1431|  EFO_0003914|               3.0|
| CHEMBL501867|MONDO_0000430|               1.0|
|   CHEMBL1168|  EFO_0000537|               4.0|
|    CHEMBL635|  EFO_0005801|               2.0|
|   CHEMBL1431|  EFO_0000313|               2.0|
|    CHEMBL716|  EFO_0002610|               4.0|
|   CHEMBL1431|  EFO_0005411|               4.0|
|CHEMBL1201589|  EFO_0007425|               3.0|
|  CHEMBL15770|  EFO_0000181|               2.0|
| CHEMBL221959|  EFO_0004192|               4.0|
|    CHEMBL480|  EFO_1001355|               4.0|
|    CHEMBL715|  EFO_0000616|               3.0|
|    CHEMBL897|MONDO_0100096|               2.0|
|CHEMBL3544911|  EFO_1000158|               1.0|
|CHEMBL3181947|MONDO

In [25]:
evidChembl.count()

48009

In [23]:
(
    indications.select(
        F.col("id").alias("drugId"),
        F.explode(F.col("approvedIndications")).alias("diseaseId"),
        F.lit("isApproved"),
    ).join(evidChembl, on=["drugId", "diseaseId"], how="right")
).filter(
    (F.col("max(clinicalPhase)") == 4) & (F.col("isApproved").isNull())
).distinct().count()

7116

In [26]:
(
    indications.select(
        F.col("id").alias("drugId"),
        F.explode(F.col("approvedIndications")).alias("diseaseId"),
        F.lit("isApproved"),
    ).join(evidChembl, on=["drugId", "diseaseId"], how="right")
).filter((F.col("max(clinicalPhase)") == 4) & (F.col("isApproved").isNull())).show()

+-------------+-------------+----------+------------------+
|       drugId|    diseaseId|isApproved|max(clinicalPhase)|
+-------------+-------------+----------+------------------+
|    CHEMBL716|  EFO_0002610|      null|               4.0|
|   CHEMBL1431|  EFO_0005411|      null|               4.0|
| CHEMBL221959|  EFO_0004192|      null|               4.0|
|    CHEMBL480|  EFO_1001355|      null|               4.0|
| CHEMBL428647|  EFO_0001645|      null|               4.0|
|CHEMBL1200480|  EFO_1001901|      null|               4.0|
|CHEMBL1096882|  EFO_0000565|      null|               4.0|
|    CHEMBL628|  EFO_0000400|      null|               4.0|
|CHEMBL1200733|MONDO_0018373|      null|               4.0|
|   CHEMBL1112|  EFO_0008623|      null|               4.0|
|    CHEMBL118|  EFO_0000731|      null|               4.0|
|    CHEMBL193|  EFO_0003917|      null|               4.0|
|    CHEMBL778|MONDO_0005180|      null|               4.0|
|CHEMBL1201581|  EFO_0004616|      null|

In [46]:
### generate the dataset for manual curation
uniqueBetas_analyse.filter(F.col("Coherency_chembl").isNotNull()).select(
    "name", "right_bio_feature"
).join(
    tissueToDisease, on=["name", "right_bio_feature"], how="outer"
).distinct().toPandas().to_csv(
    "gs://ot-team/jroldan/analysis/20240730_newcurationRBFToDisease_GOOD.csv"
)

In [47]:
uniqueBetas_analyse.filter(F.col("name") == "cardiac edema").select(
    "name", "right_bio_feature", "matchQtlRBFrelevantToDisease"
).show()

+-------------+--------------------+----------------------------+
|         name|   right_bio_feature|matchQtlRBFrelevantToDisease|
+-------------+--------------------+----------------------------+
|cardiac edema|          TFH_MEMORY|                          no|
|cardiac edema|                SKIN|                          no|
|cardiac edema|         TH17_MEMORY|                          no|
|cardiac edema|             Thyroid|                          no|
|cardiac edema|       TH1-17_MEMORY|                          no|
|cardiac edema|    Brain_Cerebellum|                          no|
|cardiac edema|CD4_T-CELL_ANTI-C...|                          no|
|cardiac edema|               BLOOD|                          no|
|cardiac edema|          TH1_MEMORY|                          no|
|cardiac edema|          TH2_MEMORY|                          no|
|cardiac edema|               BLOOD|                          no|
|cardiac edema|    SKIN_SUN_EXPOSED|                          no|
|cardiac e

In [12]:
#### select to analysis

toAnalyze = [
    "matchQtlTeTherArea",
    "matchQtlDiseaseTherArea",
    "matchQtlRBFrelevantToDisease",
    "doeCoherent",
    "doeNotCoherent",
    "doe&matchQtlTeTherArea",
    "doe&matchQtlDiseaseTherArea",
    "doe&matchQtlRBFrelevantToDisease",
    "RightTissueRightDoE",
    "RightTissueDifDoE",
    "DifTissueDifDoE",
    "DifTissueSameDoE",
]

In [27]:
uniqueBetas_analyse.withColumn(
    "RightTissueRightDoE",
    F.when(
        (F.col("matchRBFToDisease") == "yes")
        & (F.col("Coherency_chembl") == "coherent"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).withColumn(
    "RightTissueDifDoE",
    F.when(
        (F.col("matchRBFToDisease") == "yes") & (F.col("Coherency_chembl") == "dispar"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).withColumn(
    "DifTissueDifDoE",
    F.when(
        ((F.col("matchRBFToDisease") == "no") | (F.col("matchRBFToDisease").isNull()))
        & (F.col("Coherency_chembl") == "dispar"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).withColumn(
    "DifTissueSameDoE",
    F.when(
        (F.col("matchRBFToDisease") == "no")
        | (F.col("matchRBFToDisease").isNull())
        & (F.col("Coherency_chembl") == "coherent"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
    #### 3 cases of combining columns for informing safety
).withColumn(
    "SameDoERightTissueDifDoEOutTissue",
    F.when(
        (F.col("RightTissueRightDoE") == "yes") & (F.col("DifTissueDifDoE") == "yes"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).withColumn(
    "DifDoERightTissueSameDoEOutTissue",
    F.when(
        (F.col("RightTissueDifDoE") == "yes") & (F.col("DifTissueSameDoE") == "yes"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).withColumn(
    "DifDoERightTissueDifDoEOutTissue",
    F.when(
        (F.col("RightTissueDifDoE") == "yes") & (F.col("DifTissueDifDoE") == "yes"),
        F.lit("yes"),
    ).otherwise(F.lit("no")),
).groupBy(
    "matchRBFToDisease",
    "Coherency_chembl",
    "RightTissueRightDoE",
    "RightTissueDifDoE",
    "DifTissueDifDoE",
    "DifTissueSameDoE",
    #    "SameDoERightTissueDifDoEOutTissue",
    #    "DifDoERightTissueSameDoEOutTissue",
    #    "DifDoERightTissueDifDoEOutTissue",
).pivot(
    "maxClinPhase"
).count().show()

+-----------------+----------------+-------------------+-----------------+---------------+----------------+----+----+---+---+
|matchRBFToDisease|Coherency_chembl|RightTissueRightDoE|RightTissueDifDoE|DifTissueDifDoE|DifTissueSameDoE| 1.0| 2.0|3.0|4.0|
+-----------------+----------------+-------------------+-----------------+---------------+----------------+----+----+---+---+
|              yes|          dispar|                 no|              yes|             no|              no|  26| 150|  7|372|
|             null|          dispar|                 no|               no|            yes|              no|null|null| 16|  2|
|               no|          dispar|                 no|               no|            yes|             yes| 107| 148| 28|296|
|              yes|        coherent|                yes|               no|             no|              no|  59|  27|  5|506|
|               no|        coherent|                 no|               no|             no|             yes|  91| 116| 

#### collect, by target disease, if there are differences in the caregories: yes or not. 

In [29]:
aggrega = [
    "RightTissueRightDoE",
    "RightTissueDifDoE",
    "DifTissueDifDoE",
    "DifTissueSameDoE",
]

aggregated_df = (
    uniqueBetas_analyse.groupBy(
        "targetId",
        "diseaseId",
        "maxClinPhase",
    )
    .agg(*[F.collect_set(col).alias(f"agg_{col}") for col in aggrega])
    .withColumn(
        "Phase<4",
        F.when(F.col("maxClinPhase") < 4, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=3",
        F.when(F.col("maxClinPhase") <= 3, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=2",
        F.when(F.col("maxClinPhase") <= 2, F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "Phase<=1",
        F.when(F.col("maxClinPhase") <= 1, F.lit("yes")).otherwise(F.lit("no")),
    )
    .persist()
)

In [10]:
aggregated_df = (
    uniqueBetas_analyse.groupBy(
        "targetId", "diseaseId", "maxClinPhase", "right_bio_feature"
    ).agg(*[F.collect_set(col).alias(f"agg_{col}") for col in aggrega])
).show()

### falta hacer las agrupaciones y recoger que tissues estan en cada columna

NameError: name 'uniqueBetas_analyse' is not defined

In [13]:
from pyspark.sql import Window

Window.partitionBy("targetId", "diseaseId")

In [30]:
"RightTissueRightDoE",
"RightTissueDifDoE",
"DifTissueDifDoE",
"DifTissueSameDoE"

ventana = Window.partitionBy("targetId", "diseaseId")
aggregated_df = (
    (
        uniqueBetas_analyse.withColumn(
            "Tissue_rightTissuerightdoE",
            F.collect_set(
                F.when(
                    F.col("RightTissueRightDoE") == "yes", F.col("right_bio_feature")
                ).otherwise(None)
            ).over(ventana),
        )
        .withColumn(
            "Tissue_RightTissueDifDoE",
            F.collect_set(
                F.when(
                    F.col("RightTissueDifDoE") == "yes", F.col("right_bio_feature")
                ).otherwise(None)
            ).over(ventana),
        )
        .withColumn(
            "Tissue_DifTissueDifDoE",
            F.collect_set(
                F.when(
                    F.col("DifTissueDifDoE") == "yes", F.col("right_bio_feature")
                ).otherwise(None)
            ).over(ventana),
        )
        .withColumn(
            "Tissue_DifTissueSameDoE",
            F.collect_set(
                F.when(
                    F.col("RightTissueRightDoE") == "yes", F.col("right_bio_feature")
                ).otherwise(None)
            ).over(ventana),
        )
    )
    .select(
        "targetId",
        "approvedSymbol",
        "name",
        "diseaseId",
        "maxClinPhase",
        "Tissue_rightTissuerightdoE",
        "Tissue_RightTissueDifDoE",
        "Tissue_DifTissueDifDoE",
        "Tissue_DifTissueSameDoE",
    )
    .distinct()
    .show(truncate=False)
)

### has the taarget dif directionality? 

+---------------+--------------+--------------------------+-------------+------------+----------------------------------------+---------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------------------------------------+
|targetId       |approvedSymbol|name                      |diseaseId    |maxClinPhase|Tissue_rightTissuerightdoE              |Tissue_RightTissueDifDoE                                                                                 |Tissue_DifTissueDifDoE                            |Tissue_DifTissueSameDoE                 |
+---------------+--------------+--------------------------+-------------+------------+----------------------------------------+---------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------------------------------------+
|ENSG00000006071|ABCC8

In [26]:
aggregated_df.groupBy("agg_RightTissueRightDoE").pivot("Phase<=2").count().show()

+-----------------------+---+---+
|agg_RightTissueRightDoE| no|yes|
+-----------------------+---+---+
|              [no, yes]| 34| 17|
|                   [no]| 40| 27|
|                  [yes]| 13|  2|
+-----------------------+---+---+



In [27]:
aggregated_df.groupBy("agg_RightTissueDifDoE").pivot("Phase<=2").count().show()

+---------------------+----+---+
|agg_RightTissueDifDoE|  no|yes|
+---------------------+----+---+
|            [no, yes]|  30| 17|
|                 [no]|  57| 28|
|                [yes]|null|  1|
+---------------------+----+---+



In [28]:
# agg_DifTissueDifDoE
aggregated_df.groupBy("agg_DifTissueDifDoE").pivot("Phase<=2").count().show()

+-------------------+---+---+
|agg_DifTissueDifDoE| no|yes|
+-------------------+---+---+
|          [no, yes]| 40| 24|
|               [no]| 37| 17|
|              [yes]| 10|  5|
+-------------------+---+---+



In [ ]:
aggregated_df = (
    df.groupBy("targetId", "diseaseId")
    .agg(
        *[
            F.collect_set(
                F.when(F.col(col) == "yes", F.col(col_name)).otherwise(None)
            ).alias(f"unique_{col_name}")
            for col_name in toAnalyze
        ]
    )
    .persist()
)

In [ ]:
uniqueBetas_analyse.groupBy("targetId", "diseaseId").agg(F.collect_set)

In [9]:
aggregated_df.show()

+---------------+-------------+-------------------------+------------------------------+-----------------------------------+------------------+---------------------+-----------------------------+----------------------------------+---------------------------------------+--------------------------+------------------------+----------------------+-----------------------+
|       targetId|    diseaseId|unique_matchQtlTeTherArea|unique_matchQtlDiseaseTherArea|unique_matchQtlRBFrelevantToDisease|unique_doeCoherent|unique_doeNotCoherent|unique_doe&matchQtlTeTherArea|unique_doe&matchQtlDiseaseTherArea|unique_doe&matchQtlRBFrelevantToDisease|unique_RightTissueRightDoE|unique_RightTissueDifDoE|unique_DifTissueDifDoE|unique_DifTissueSameDoE|
+---------------+-------------+-------------------------+------------------------------+-----------------------------------+------------------+---------------------+-----------------------------+----------------------------------+------------------------------